In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [2]:
def CountFrequency(my_list): 
    freq = {} 
    for item in my_list: 
        if (item in freq): 
            freq[item] += 1
        else: 
            freq[item] = 1
  
    return freq 

In [3]:
def default_clean(x):
    if type(x) is str:
        x = x.lower()
        x = re.sub('<[^<]+?>', ' ', x)
        x = re.sub(r'^https?:\/\/.*[\r\n]*', ' ', x, flags=re.MULTILINE)
        x = re.sub('[^A-Z a-z]+', ' ', x)
        return x
    else:
        return x

In [4]:
def stop_and_stem(text, stem=True, stemmer = PorterStemmer()):
    '''
    Removes stopwords and does stemming
    '''
    stoplist = stopwords.words('english')
    if stem:
        text_stemmed = [stemmer.stem(word) for word in word_tokenize(text) if word not in stoplist ]
    else:
        text_stemmed = [word for word in word_tokenize(text) if word not in stoplist ]
    text = ' '.join(text_stemmed)
    return text

In [5]:
def test_predict(i):
    rand_int = i
    test_sample = sample.iloc[rand_int]['train_feature']
    label = sample.iloc[rand_int, sample.columns.get_loc('main_cat')] 
    
    test_sample = default_clean(test_sample)
    test_sample = stop_and_stem(test_sample, stem=False)
    
    new_doc_words = test_sample.split()
    new_doc_vec = model.infer_vector(new_doc_words, steps=50, alpha=0.25) 
    
    similars = model.docvecs.most_similar(positive=[new_doc_vec])
    return label,similars

In [6]:
sample = pd.read_json("Meta_v4.json")

In [7]:
for i in range(len(sample)):
    if sample['specifications'][i]=={}:
        sample = sample.drop(i)

In [8]:
sample=sample.reset_index()
sample = sample.drop(['index'],axis=1)

In [9]:
%%time
for i in range(len(sample['description'])):
    sample['description'][i] = ' '.join(map(str,sample['description'][i])) 
    
for i in range(len(sample['feature'])):
    sample['feature'][i] = ' '.join(map(str,sample['feature'][i])) 

Wall time: 2.34 s


In [10]:
sample['train_feature'] = sample['description'] + ' ' + sample['title'] + ' ' + sample['feature']

In [11]:
sample = sample.dropna()

In [12]:
sample = sample[sample['description'] != '']

In [13]:
sample['main_cat']= sample['main_cat'].replace('Home Audio &amp; Theater', 'Home Audio & Theater')
sample['main_cat']= sample['main_cat'].replace('Car Electronics', 'All Electronics')
sample['main_cat']= sample['main_cat'].replace('Portable Audio & Accessories', 'All Electronics')
sample['main_cat']= sample['main_cat'].replace('Cell Phones & Accessories', 'All Electronics')
sample['main_cat']= sample['main_cat'].replace('Cell Phones &amp; Accessories', 'All Electronics')
sample['main_cat']= sample['main_cat'].replace('Tools & Home Improvement', 'All Electronics')
sample['main_cat']= sample['main_cat'].replace('Tools &amp; Home Improvement', 'All Electronics')
sample['main_cat']= sample['main_cat'].replace('GPS & Navigation', 'All Electronics')
sample['main_cat']= sample['main_cat'].replace('GPS &amp; Navigation', 'All Electronics')
sample['main_cat']= sample['main_cat'].replace('Tools &amp; Home Improvement', 'All Electronics')
sample['main_cat']= sample['main_cat'].replace('Camera &amp; Photo', 'Camera & Photo')

In [14]:
frequency = CountFrequency(sample['main_cat'])

In [15]:
frequency

{'Home Audio & Theater': 610,
 'Office Products': 111,
 'Camera & Photo': 532,
 'Computers': 2802,
 'All Electronics': 927,
 'Musical Instruments': 21,
 'Automotive': 43,
 'Amazon Home': 19,
 'Amazon Devices': 6,
 'All Beauty': 3,
 '': 2,
 'Sports & Outdoors': 2,
 'Industrial & Scientific': 2,
 'Health & Personal Care': 3,
 '<img src="https://images-na.ssl-images-amazon.com/images/G/01/nav2/images/gui/amazon-fashion-store-new._CB520838675_.png" class="nav-categ-image" alt="AMAZON FASHION"/>': 1,
 'Arts, Crafts & Sewing': 1}

In [16]:
removal_list = ['Amazon Devices',
                'Baby',
                'All Beauty',
                'Toys & Games',
                'Sports & Outdoors',
                '',
                'Apple Products',
                'Health & Personal Care',
                'Industrial & Scientific',
                'Arts, Crafts & Sewing',
                '<img src="https://images-na.ssl-images-amazon.com/images/G/01/nav2/images/gui/amazon-fashion-store-new._CB520838675_.png" class="nav-categ-image" alt="AMAZON FASHION"/>']

In [17]:
for i in range(len(removal_list)):
    sample = sample[sample['main_cat'] != removal_list[i]]

In [18]:
sample['train_feature'] = sample['train_feature'].apply(default_clean)
sample['train_feature'] = sample['train_feature'].apply(stop_and_stem, stem=False)

In [19]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import numpy as np

[nltk_data] Downloading package punkt to C:\Users\Atul
[nltk_data]     Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
sentences = sample['train_feature'].tolist()

In [21]:
tokenized_sent = []
for s in sentences:
    tokenized_sent.append(word_tokenize(s.lower()))

In [22]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_sent)]
tagged_data

[TaggedDocument(words=['true', 'high', 'definition', 'sound', 'cvc', 'noise', 'cancellation', 'technology', 'safari', 'bluetooth', 'headphone', 'intelligently', 'filters', 'background', 'noise', 'sound', 'transmission', 'busy', 'noisy', 'environments', 'latest', 'bluetooth', 'technology', 'features', 'faster', 'stable', 'signal', 'clearer', 'sound', 'quality', 'lower', 'power', 'consumption', 'sweat', 'proof', 'protection', 'ipx', 'water', 'resistant', 'material', 'prevents', 'headphone', 'damage', 'getting', 'drenched', 'sweat', 'grueling', 'workout', 'long', 'battery', 'life', 'hours', 'continuous', 'music', 'talk', 'time', 'hours', 'standby', 'single', 'charge', 'spend', 'time', 'listening', 'music', 'less', 'time', 'charging', 'stylish', 'ergonomic', 'design', 'choice', 'small', 'medium', 'large', 'size', 'ear', 'tips', 'bound', 'secure', 'fit', 'stays', 'place', 'whether', 'running', 'working', 'universally', 'compatible', 'works', 'bluetooth', 'enabled', 'devices', 'wireless', 'b

In [23]:
## Train doc2vec model
model = Doc2Vec(tagged_data, vector_size = 20, window = 2, min_count = 1, epochs = 100)

'''
vector_size = Dimensionality of the feature vectors.
window = The maximum distance between the current and predicted word within a sentence.
min_count = Ignores all words with total frequency lower than this.
alpha = The initial learning rate.
'''
model.save('Product_Similarity_Model.doc2vec')

In [26]:
category = sample['main_cat'].tolist()

In [27]:
count = 0
score = []
embedding_matrix = []
for i in range(len(sentences)):
    count = 0
    test_doc = word_tokenize(sentences[i].lower())
    test_doc_vector = model.infer_vector(test_doc)
    embedding_matrix.append(test_doc_vector)
    model.docvecs.most_similar([test_doc_vector])
    similar_sentences = len(model.docvecs.most_similar([test_doc_vector]))
    for j in range(similar_sentences):
        index = model.docvecs.most_similar([test_doc_vector])[j][0]
        if category[index] == category[i]:
            count = count + 1
    score.append(count/similar_sentences)

0 1.0
1 0.2
2 0.9
3 0.9
4 1.0
5 0.3
6 0.8
7 0.2
8 1.0
9 1.0
10 0.7
11 0.8
12 0.7
13 1.0
14 0.3
15 0.6
16 1.0
17 0.9
18 0.7
19 1.0
20 1.0
21 0.4
22 1.0
23 0.2
24 0.2
25 0.8
26 1.0
27 0.8
28 1.0
29 0.9
30 0.9
31 1.0
32 0.5
33 1.0
34 1.0
35 0.6
36 0.2
37 1.0
38 1.0
39 1.0
40 0.8
41 0.6
42 1.0
43 1.0
44 1.0
45 0.9
46 0.1
47 1.0
48 1.0
49 1.0
50 1.0
51 0.5
52 0.8
53 0.1
54 1.0
55 0.9
56 0.9
57 0.6
58 0.9
59 0.2
60 1.0
61 0.8
62 0.6
63 1.0
64 0.8
65 0.9
66 0.9
67 0.2
68 1.0
69 0.7
70 0.8
71 0.3
72 0.7
73 1.0
74 0.4
75 0.9
76 1.0
77 0.8
78 0.4
79 0.3
80 0.3
81 0.3
82 1.0
83 0.3
84 0.5
85 0.3
86 0.9
87 0.5
88 0.3
89 0.3
90 0.3
91 0.8
92 1.0
93 1.0
94 0.8
95 0.5
96 1.0
97 1.0
98 0.8
99 0.9
100 1.0
101 1.0
102 0.7
103 0.1
104 1.0
105 1.0
106 0.9
107 1.0
108 0.5
109 1.0
110 0.9
111 0.9
112 1.0
113 1.0
114 0.2
115 0.1
116 0.7
117 0.4
118 0.5
119 1.0
120 1.0
121 0.1
122 0.8
123 0.2
124 0.5
125 0.1
126 0.7
127 0.2
128 0.8
129 0.9
130 1.0
131 1.0
132 0.5
133 0.5
134 0.1
135 0.1
136 1.0
137 1.0
138 1.

1034 1.0
1035 0.9
1036 1.0
1037 0.9
1038 1.0
1039 1.0
1040 0.5
1041 1.0
1042 0.9
1043 0.2
1044 1.0
1045 1.0
1046 1.0
1047 0.8
1048 0.3
1049 0.6
1050 0.2
1051 1.0
1052 1.0
1053 0.1
1054 0.2
1055 1.0
1056 0.4
1057 0.7
1058 1.0
1059 0.3
1060 1.0
1061 1.0
1062 0.8
1063 0.6
1064 1.0
1065 1.0
1066 0.7
1067 1.0
1068 1.0
1069 1.0
1070 1.0
1071 1.0
1072 0.3
1073 0.3
1074 0.2
1075 0.9
1076 1.0
1077 1.0
1078 0.6
1079 1.0
1080 1.0
1081 0.9
1082 1.0
1083 0.7
1084 0.6
1085 1.0
1086 1.0
1087 0.9
1088 0.9
1089 1.0
1090 1.0
1091 1.0
1092 1.0
1093 0.6
1094 1.0
1095 0.2
1096 1.0
1097 1.0
1098 0.8
1099 0.3
1100 1.0
1101 0.3
1102 1.0
1103 0.5
1104 1.0
1105 0.8
1106 1.0
1107 1.0
1108 1.0
1109 1.0
1110 1.0
1111 0.7
1112 0.1
1113 1.0
1114 1.0
1115 1.0
1116 1.0
1117 0.7
1118 1.0
1119 1.0
1120 0.8
1121 0.7
1122 0.3
1123 1.0
1124 0.9
1125 0.9
1126 0.7
1127 1.0
1128 0.9
1129 0.8
1130 0.2
1131 0.1
1132 0.2
1133 0.8
1134 1.0
1135 1.0
1136 0.1
1137 1.0
1138 1.0
1139 1.0
1140 0.1
1141 0.9
1142 1.0
1143 0.9
1144 1.0
1

1947 0.1
1948 1.0
1949 1.0
1950 0.7
1951 1.0
1952 0.3
1953 0.2
1954 1.0
1955 0.4
1956 0.6
1957 0.6
1958 1.0
1959 1.0
1960 1.0
1961 0.9
1962 0.9
1963 0.8
1964 0.1
1965 0.6
1966 0.4
1967 0.6
1968 0.6
1969 0.4
1970 0.6
1971 0.4
1972 1.0
1973 0.9
1974 1.0
1975 0.4
1976 0.1
1977 0.4
1978 1.0
1979 1.0
1980 0.8
1981 0.9
1982 1.0
1983 0.8
1984 1.0
1985 1.0
1986 0.1
1987 1.0
1988 0.9
1989 0.8
1990 0.1
1991 0.3
1992 1.0
1993 0.9
1994 0.8
1995 0.5
1996 1.0
1997 1.0
1998 1.0
1999 0.2
2000 0.9
2001 1.0
2002 1.0
2003 0.2
2004 1.0
2005 0.9
2006 0.9
2007 0.7
2008 1.0
2009 1.0
2010 0.9
2011 1.0
2012 0.4
2013 0.1
2014 0.8
2015 0.7
2016 0.3
2017 0.8
2018 1.0
2019 0.6
2020 0.6
2021 1.0
2022 0.8
2023 1.0
2024 0.8
2025 1.0
2026 0.6
2027 0.8
2028 1.0
2029 0.4
2030 1.0
2031 1.0
2032 0.3
2033 0.1
2034 1.0
2035 0.3
2036 1.0
2037 0.6
2038 0.9
2039 0.4
2040 0.9
2041 0.9
2042 1.0
2043 0.5
2044 0.9
2045 0.8
2046 0.8
2047 0.8
2048 1.0
2049 0.9
2050 0.8
2051 0.7
2052 1.0
2053 0.5
2054 1.0
2055 0.2
2056 0.4
2057 0.4
2

2858 0.6
2859 0.5
2860 0.8
2861 0.9
2862 0.8
2863 0.7
2864 0.9
2865 0.9
2866 1.0
2867 1.0
2868 1.0
2869 0.9
2870 0.4
2871 0.2
2872 0.3
2873 0.0
2874 1.0
2875 1.0
2876 1.0
2877 1.0
2878 0.5
2879 0.5
2880 1.0
2881 1.0
2882 0.4
2883 0.3
2884 0.1
2885 1.0
2886 0.9
2887 0.1
2888 1.0
2889 1.0
2890 0.9
2891 0.8
2892 1.0
2893 1.0
2894 0.4
2895 1.0
2896 1.0
2897 0.8
2898 0.4
2899 0.9
2900 0.4
2901 0.4
2902 0.6
2903 0.8
2904 0.9
2905 1.0
2906 1.0
2907 0.9
2908 1.0
2909 1.0
2910 0.5
2911 0.3
2912 0.2
2913 0.2
2914 1.0
2915 0.3
2916 0.2
2917 0.4
2918 1.0
2919 0.6
2920 1.0
2921 0.3
2922 0.7
2923 0.8
2924 1.0
2925 0.1
2926 1.0
2927 1.0
2928 1.0
2929 0.4
2930 0.2
2931 0.9
2932 0.6
2933 0.7
2934 0.7
2935 1.0
2936 1.0
2937 1.0
2938 1.0
2939 0.9
2940 1.0
2941 1.0
2942 1.0
2943 1.0
2944 0.6
2945 0.8
2946 0.2
2947 1.0
2948 1.0
2949 0.4
2950 1.0
2951 0.9
2952 1.0
2953 0.4
2954 1.0
2955 1.0
2956 0.6
2957 1.0
2958 1.0
2959 0.2
2960 1.0
2961 1.0
2962 1.0
2963 0.8
2964 0.5
2965 0.6
2966 0.5
2967 1.0
2968 1.0
2

3774 0.8
3775 0.9
3776 0.7
3777 0.9
3778 1.0
3779 1.0
3780 1.0
3781 0.8
3782 1.0
3783 0.6
3784 1.0
3785 0.4
3786 0.6
3787 1.0
3788 0.8
3789 0.9
3790 0.8
3791 0.9
3792 0.7
3793 0.6
3794 1.0
3795 0.5
3796 0.1
3797 0.9
3798 1.0
3799 0.9
3800 0.7
3801 0.2
3802 1.0
3803 1.0
3804 0.9
3805 1.0
3806 1.0
3807 0.9
3808 0.9
3809 0.8
3810 0.9
3811 1.0
3812 1.0
3813 0.6
3814 1.0
3815 0.5
3816 1.0
3817 0.2
3818 0.8
3819 0.7
3820 1.0
3821 0.9
3822 0.9
3823 1.0
3824 0.4
3825 0.4
3826 1.0
3827 0.8
3828 1.0
3829 1.0
3830 0.7
3831 0.4
3832 0.7
3833 0.7
3834 1.0
3835 0.3
3836 0.9
3837 1.0
3838 1.0
3839 1.0
3840 0.9
3841 0.2
3842 0.2
3843 1.0
3844 0.7
3845 1.0
3846 0.7
3847 1.0
3848 0.3
3849 0.5
3850 0.7
3851 0.1
3852 0.7
3853 0.9
3854 0.8
3855 0.1
3856 0.2
3857 1.0
3858 0.7
3859 0.7
3860 1.0
3861 1.0
3862 0.2
3863 0.1
3864 0.9
3865 1.0
3866 0.6
3867 0.6
3868 0.2
3869 0.6
3870 0.9
3871 1.0
3872 0.6
3873 0.8
3874 1.0
3875 1.0
3876 0.9
3877 0.8
3878 0.7
3879 1.0
3880 0.9
3881 0.7
3882 0.8
3883 0.9
3884 0.3
3

4687 1.0
4688 0.1
4689 1.0
4690 0.2
4691 0.9
4692 0.9
4693 0.7
4694 0.3
4695 0.9
4696 0.4
4697 0.5
4698 0.2
4699 1.0
4700 0.6
4701 1.0
4702 0.9
4703 1.0
4704 0.7
4705 0.4
4706 0.5
4707 0.5
4708 1.0
4709 0.9
4710 0.9
4711 1.0
4712 0.8
4713 0.5
4714 1.0
4715 0.2
4716 1.0
4717 1.0
4718 0.6
4719 0.7
4720 1.0
4721 1.0
4722 0.5
4723 1.0
4724 0.8
4725 0.4
4726 0.9
4727 1.0
4728 0.5
4729 0.6
4730 0.9
4731 1.0
4732 0.8
4733 0.2
4734 0.7
4735 0.6
4736 0.7
4737 0.6
4738 1.0
4739 0.7
4740 1.0
4741 0.9
4742 1.0
4743 1.0
4744 1.0
4745 1.0
4746 1.0
4747 0.7
4748 0.5
4749 0.9
4750 0.9
4751 0.9
4752 0.9
4753 1.0
4754 0.1
4755 1.0
4756 1.0
4757 1.0
4758 1.0
4759 1.0
4760 1.0
4761 0.4
4762 0.8
4763 0.6
4764 1.0
4765 1.0
4766 1.0
4767 1.0
4768 1.0
4769 1.0
4770 1.0
4771 0.9
4772 1.0
4773 1.0
4774 0.8
4775 0.9
4776 0.6
4777 0.3
4778 0.1
4779 0.5
4780 0.3
4781 1.0
4782 0.8
4783 0.5
4784 1.0
4785 1.0
4786 0.9
4787 0.4
4788 0.7
4789 0.4
4790 0.6
4791 0.6
4792 1.0
4793 0.4
4794 0.1
4795 1.0
4796 1.0
4797 1.0
4

In [28]:
sample['similarity_score'] = score
sample['embedding_vectors'] = embedding_matrix

In [29]:
sample

,also_buy,also_view,asin,brand,category,date,description,feature,image,main_cat,price,rank,title,specifications,train_feature,similarity_score,embedding_vectors
0,[],[],0132492776,Enter The Arena,"[Electronics, Headphones, Earbud Headphones]","October 23, 2017",<b>True High Definition Sound:</b><br>With CV...,Superb Sound Quality: Plays crystal clear audi...,[https://images-na.ssl-images-amazon.com/image...,Home Audio & Theater,$7.99,[>#950 in Cell Phones & Accessories (See Top 1...,Wireless Bluetooth Headphones Earbuds with Mic...,{'waterproof_rating': 'IPX7'},true high definition sound cvc noise cancellat...,1.0,"[-0.34834507, 2.3380778, -0.18207246, 2.753408..."
1,"[1929854439, B00KY1DZJ8, 128413699X, B00H5A36Y...","[B00H0QP0MA, B00KY1DZJ8, B01AX6NOTE, 192985443...",0692056343,Hospital Procedures Consultants,"[Electronics, Accessories &amp; Supplies, Offi...","January 16, 2018",A must-have for any medical professional spend...,The most comprehensive and reliable bedside em...,[https://images-na.ssl-images-amazon.com/image...,Office Products,,"[>#290,628 in Office Products (See top 100), >...",Emergency and Critical Care Drug and Medicatio...,{'Item Dimensions': '6 x 4 x 0.1 inches'},must medical professional spending time er icu...,0.2,"[3.0337863, -4.4503, -4.5237403, 0.070967235, ..."
2,[],[],B019W8R66G,Abowone,"[Electronics, Camera & Photo, Video Surveillan...","December 21, 2017",Product Description Style Bullet IP Camera wit...,"3.6mm lens, HD 1920x 1080p 2 Megapixel ,IP66 w...",[https://images-na.ssl-images-amazon.com/image...,Camera & Photo,,"[>#34,590 in Camera & Photo (See Top 100 in Ca...",Abowone 2.0MP 1080P Bullet IP Camera with PoE ...,"{'screen_resolution': '1920x 1080', 'camera_re...",product description style bullet ip camera poe...,0.9,"[-3.1358848, -0.6637256, -2.1783204, 1.1176518..."
5,[],"[B07D75SQWY, B07KF849NX, B00XHSKKXE, B07GFDFNY...",B015Q8QGG2,Asobeage,"[Electronics, Camera & Photo, Video]","November 11, 2018","Asobeage DOES NOT join the price war.Instead,w...",Get the First Person Point of View Easily and ...,[https://images-na.ssl-images-amazon.com/image...,Camera & Photo,$27.99,"[>#19,819 in Camera & Photo (See Top 100 in Ca...",Asobeage 50-in-1 Gopro Accessories Kit for Her...,{'sensitivity': ' DB'},asobeage join price war instead focus product ...,0.9,"[-1.3052427, -0.8287602, -0.51811975, -1.47482..."
6,[],"[B075RZJS2Z, B07HH7L6Z8, B07FVHJTFL, B015Z7XE0...",B015QBPK6Q,burkemany,"[Electronics, Computers & Accessories, Tablet ...","December 5, 2018",<b>Feature:</b><br> Double sides of adapter ca...,This Usb Type C to Magsafe power adapter item ...,[https://images-na.ssl-images-amazon.com/image...,Computers,$19.99,[>#70 in Computers & Accessories > Tablet Acce...,Compatible MagSafe USB Type C Converter Replac...,"{'Size': '15""'}",feature double sides adapter work double sides...,1.0,"[-1.2369837, -0.4991285, 1.2082498, 2.7016304,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6895,[],"[B06XD3LXXK, B0759YSF4W]",B01CUJCQA0,Acer,"[Electronics, Computers & Accessories, Compute...","March 23, 2017",<br>Acer C720-2103 Chromebook comes with these...,This Certified Refurbished product is tested a...,[https://images-na.ssl-images-amazon.com/image...,Computers,,"[>#93,687 in Computers & Accessories (See top ...","2017 Acer C720-2103 Chromebook (11.6-Inch, Int...","{'Screen Size': '1 inches', 'Processor': '1.4 ...",acer c chromebook comes high level specs intel...,1.0,"[0.2241844, -1.3417784, -2.0426245, 0.3368168,..."
6898,[],[],B015PKMGRE,Flashmen,"[Electronics, Computers & Accessories, Compute...","May 19, 2017",<b>Description:</b><br> It is a better choise ...,"Standard Cat6 Ethernet Cable, backward compati...",[https://images-na.ssl-images-amazon.com/image...,Computers,$14.99,[],Flashmen RJ45 Cat6 Ethernet Patch Cable 24AGW ...,"{'speed': '550MHz', 'transfer_speed': ' 10Gbps...",description better choise cat ethernet cable f...,1.0,"[-0.7482374, -2.5132675, 3.2881372, 3.

In [30]:
sample.to_csv('scored_dataset.csv')